In [ ]:
import os
import time
import copy
import random
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from PIL import Image
from torch.utils.data import Subset
from torch.optim import lr_scheduler
from torchvision import transforms,datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained("efficientnet-b0")
model_name = "efficientnet-b0"

# data 불러오기
- transforms.Compose 안의 Resize, ToTensor, Normalize 기능으로 다양한 크기의 이미지들을 한번에 정돈하여 불러옵니다.
- mageNet은 입력이 224x224 형식이므로 이에 맞춰 Resize 해줍니다.  torch의 입력형태인 Tensor로 바꿔 준 후 Normalize 해줍니다.
- sklearn의 train_test_split 함수로 전체 데이터를 train/valid/test 셋으로 나눠줍니다. (8:1:1)

In [ ]:
batch_size = 256
random.seed(42)
torch.manual_seed(42)
print(os.getcwd())  # 현재 작업 디렉토리를 출력합니다.

data_path= "/content/drive/MyDrive/9. newdata/nnnnewdata"
# ImageFolder 함수를 사용해 이미지 데이터셋을 불러오며, Compose를 사용해 이미지에 변환작업 진행.
car_dataset = datasets.ImageFolder(data_path,
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
    ]))
train_idx, tmp_idx = train_test_split(list(range(len(car_dataset))), test_size=0.2, random_state=random_seed)
datasets = {}
datasets['train'] = Subset(car_dataset, train_idx)
tmp_dataset       = Subset(car_dataset, tmp_idx)

val_idx, test_idx = train_test_split(list(range(len(tmp_dataset))), test_size=0.5, random_state=random_seed)
datasets['valid'] = Subset(tmp_dataset, val_idx)
datasets['test']  = Subset(tmp_dataset, test_idx)

## data loader
dataloaders, batch_num = {}, {}
dataloaders['train'] = torch.utils.data.DataLoader(datasets['train'],
                                                batch_size=batch_size, shuffle=True,
                                                num_workers=4)
dataloaders['valid'] = torch.utils.data.DataLoader(datasets['valid'],
                                                batch_size=batch_size, shuffle=False,
                                                num_workers=4)
dataloaders['test']  = torch.utils.data.DataLoader(datasets['test'],
                                                batch_size=batch_size, shuffle=False,
                                                num_workers=4)
batch_num['train'], batch_num['valid'], batch_num['test'] = len(dataloaders['train']), len(dataloaders['valid']), len(dataloaders['test'])
print('batch_size : %d,  tvt : %d / %d / %d' % (batch_size, batch_num['train'], batch_num['valid'], batch_num['test']))

# data 확인
- dataset.ImagFolder를 잘 가져 오는지 확인합니다.
- transfoms로 tensor로 변환된 이미지를 불러와 class이름과 같이 확인 하는 작업을 진행합니다.

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))  # 텐서를 넘파이 배열로 변환하고, 축을 변경합니다.
    mean = np.array([0.485, 0.456, 0.406])  # 정규화를 해제하기 위한 평균 값입니다.
    std = np.array([0.229, 0.224, 0.225])   # 정규화를 해제하기 위한 표준편차 값입니다.
    inp = std * inp + mean  # 정규화를 해제합니다.
    inp = np.clip(inp, 0, 1)  # 이미지 배열의 값이 0과 1 사이의 값이 되도록 합니다.
    plt.imshow(inp)  # 이미지를 출력합니다.
    if title is not None:  # 만약 타이틀이 있다면,
        plt.title(title)  # 그 타이틀을 출력합니다.
    plt.pause(0.001)  # 그림이 업데이트 되는 것을 잠시 멈춥니다.

num_show_img = 5  # 보여줄 이미지의 수를 설정합니다.
class_names={   # 클래스 이름을 사전형으로 설정합니다.
    "00":"MAXCRUZ",
    "01":"MORNING",
    "02":"SONATA",
    "03":"TIVOLI",
    "04":"SM5",
    "05":"STINGER",
    "06":"MOHAVE",
    "07":"CARNIVAL",
    "08":"TUCSON",
    "09":"CLIO"
}
# train, valid, test 이미지를 가져와 그디르 형태로 만듭니다.
# train
inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs[:num_show_img])
imshow(out, title=[class_names[str(int(x))] for x in classes[:num_show_img]])
# valid
inputs, classes = next(iter(dataloaders['valid']))
out = torchvision.utils.make_grid(inputs[:num_show_img])
imshow(out, title=[class_names[str(int(x))] for x in classes[:num_show_img]])
# test
inputs, classes = next(iter(dataloaders['test']))
out = torchvision.utils.make_grid(inputs[:num_show_img])
imshow(out, title=[class_names[str(int(x))] for x in classes[:num_show_img]])

# 학습 코드 작성
- epoch을 돌때 마다 훈련하고, 평가를 진행하면서 평가 정확도가 높으면 저장합니다.

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    # 학습 시작 시간을 기록
    since = time.time()
    # 최적의 모델 가중치를 저장할 변수를 초기화합니다.
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_loss, train_acc, valid_loss, valid_acc = [], [], [], []
    # 지정된 에포크만큼 반복하여 학습을 수행합니다.
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # 각 에포크에서는 학습 단계와 검증 단계를 모두 실행합니다.
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # 학습 모드로 모델을 설정합니다.
            else:
                model.eval()   # 검증 모드로 모델을 설정합니다.
            running_loss, running_corrects, num_cnt = 0.0, 0, 0
            # 데이터 로더에서 배치 단위로 데이터를 가져와 처리합니다.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # 모델의 파라미터 그라디언트를 초기화합니다.
                optimizer.zero_grad()
                # 학습 단계에서만 그라디언트를 계산하도록 설정합니다.
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)  # 모델의 예측값을 계산합니다.
                    _, preds = torch.max(outputs, 1)  # 예측값 중 가장 높은 확률을 가진 클래스를 선택합니다.
                    loss = criterion(outputs, labels)  # 손실을 계산합니다.
                    # 학습 단계에서는 역전파를 수행하고, 옵티마이저를 이용해 파라미터를 업데이트합니다.
                    if phase == 'train':
                        loss.backward()  # 역전파를 수행합니다.
                        optimizer.step()  # 옵티마이저를 이용해 파라미터를 업데이트합니다.
                # 통계를 계산합니다.
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += len(labels)
            if phase == 'train':
                scheduler.step()
            epoch_loss = float(running_loss / num_cnt)
            epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)
            # 학습과 검증 단계에서의 손실과 정확도를 기록합니다.
            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                valid_loss.append(epoch_loss)
                valid_acc.append(epoch_acc)
            print('{} Loss: {:.2f} Acc: {:.1f}'.format(phase, epoch_loss, epoch_acc))
            # 검증 단계에서 정확도가 이전보다 좋을 경우, 모델의 상태를 저장합니다.
            if phase == 'valid' and epoch_acc > best_acc:
                best_idx = epoch
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print('==> best model saved - %d / %.1f'%(best_idx, best_acc))

    # 학습에 걸린 시간을 계산하고 출력합니다.
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: %d - %.1f' %(best_idx, best_acc))
    # 가장 성능이 좋았던 모델의 상태를 불러옵니다.
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'car_model.pt')  # 모델을 저장합니다.
    print('model saved')
    return model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc  # 학습 결과를 반환합니다.

# 설정
- GPU를 사용한다고 선언합니다.
- 분류 문제이므로 CrossEntropy를 설정합니다. (Negative log를 씌운걸 쓰기도 하는데, SGD, RMS를 사용한 결과 SGD가 잘 작동하는것을 확인)
- Lr를 미세하게 조정하여 최적의 값을 찾는다.

In [ ]:
# CUDA를 사용할 수 있는 경우 GPU를, 그렇지 않은 경우 CPU를 사용하도록 설정합니다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  
# 모델을 해당 장치로 이동시킵니다.
model = model.to(device)
# 손실 함수로 CrossEntropyLoss를 사용합니다.
criterion = nn.CrossEntropyLoss()
# 옵티마이저로 Stochastic Gradient Descent (SGD)를 사용하며, 학습률은 0.01, 모멘텀은 0.9, 가중치 감쇠는 1e-4로 설정합니다.
optimizer_ft = optim.SGD(model.parameters(),
                        lr = 0.01,
                        momentum=0.9,
                        weight_decay=1e-4)
# 학습률 스케줄러를 설정합니다. 각 에포크마다 학습률을 0.98739로 곱하여 학습률을 감소시킵니다.
lmbda = lambda epoch: 0.98739
exp_lr_scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer_ft, lr_lambda=lmbda)

# 학습 진행

In [ ]:
# train_model 함수를 이용해 모델을 학습시킵니다. 이 함수는 학습된 모델, 가장 좋은 성능을 낸 에포크의 인덱스, 
# 그 때의 정확도, 각 에포크마다의 학습 손실과 정확도, 검증 손실과 정확도를 반환합니다.
model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)
